In [17]:
import geopandas as gpd
from odc.geo import XY, BoundingBox
from odc.geo.gridspec import GridSpec

In [18]:
# Create a gridspec
pacific_epsg = "EPSG:3832"
# The origin is in the projected CRS. This works for Landsat.
pacific_gridspec = GridSpec(crs=pacific_epsg, tile_shape=(3200,3200), resolution=30, origin=XY(-3000000,-4000000))

# This grid is for Sentinel-2 and has the same footprint
# pacific_gridspec_s2 = GridSpec(crs=pacific_epsg, tile_shape=(9600,9600), resolution=10, origin=XY(-3000000,-4000000))

In [19]:
# Select relevant tiles and convert to Geopandas
bounds = BoundingBox(120, -30, 280, 30, crs="EPSG:4326").to_crs(pacific_epsg)
geom = pacific_gridspec.geojson(bbox=bounds)

for feature in geom['features']:
    new_coords = []
    for coordinate in feature['geometry']['coordinates']:
        for coordinate_pair in coordinate:
            if coordinate_pair[0] < 0:
                coordinate_pair = (coordinate_pair[0] + 360, coordinate_pair[1])
            new_coords.append(coordinate_pair)
    feature['geometry']['coordinates'] = [new_coords]

tiles = gpd.GeoDataFrame.from_features(geom, crs="EPSG:4326")

In [20]:
# Get the Pacific admin boundaries
# These need to be downloaded from ... can't remember where
pacific_admin = gpd.read_file("pacific_admin.gpkg")

In [26]:
# Select tiles that intersect with the Pacific admin boundaries
tiles_projected = tiles.to_crs(pacific_epsg)
admin_projected = pacific_admin.to_crs(pacific_epsg)

# Buffering takes a long time... let's do this step separately
admin_buffer_50k = admin_projected.buffer(50_000)


In [40]:
admin_buffer_50k = gpd.GeoDataFrame(admin_projected, geometry=admin_buffer_50k ,crs=pacific_epsg)

In [33]:
type(tiles_projected)

geopandas.geodataframe.GeoDataFrame

In [41]:
intersected = gpd.sjoin(tiles_projected, admin_buffer_50k, how="inner", predicate="intersects")

# Pick the first country name for each tile... one country wins, another is lost to time...
intersected = intersected.groupby("idx").agg({"ISO2": ','.join, "geometry": "first"})

In [43]:
renamed = intersected.rename(columns={"ISO2": "country_code"})
renamed["bbox"] = renamed["geometry"].apply(lambda x: ",".join([f"{b:6.0f}" for b in x.bounds]))
# Drop the geometry from renamed
renamed = renamed.drop(columns=["geometry"])

joined = tiles.join(renamed, how="inner", on="idx")
joined = joined.rename(columns={"idx": "tile_id"})

In [44]:
joined.explore()

In [45]:
joined.to_file("grid_pacific.geojson")